In [2]:
import os
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import generate_imgs_for_training

# make directory 
dirname = 'img_data'
if os.path.exists(dirname): shutil.rmtree(dirname) # remove if exists
if not os.path.exists(dirname): os.makedirs(dirname)

In [ ]:
# set participant id
df_all = pd.DataFrame
for pid in ["559", "563", "570", "575", "588", "591"]:

    # read data
    df = pd.read_csv(f"../../datasets/processed/{pid}-train.csv")
    df[["iob_raw", "basal_raw", "bolus_raw"]] = df[["iob_raw", "basal_raw", "bolus_raw"]].apply(lambda col: col.map(lambda x: 0 if x < 0 else x))
    df = df.dropna(axis=0)

    # discretize basal and bolus
    bins_for_basal_bounds = np.linspace(-0.1, 5.1, 257) # 256 - upper bound is 5
    bins_for_bolus_bounds = np.linspace(-0.1, 25.1, 257) # 256 - upper bound is 25

    df['basal_binned'] = pd.cut(df['basal_raw'], bins=bins_for_basal_bounds, labels=False)
    df['bolus_binned'] = pd.cut(df['bolus_raw'], bins=bins_for_bolus_bounds, labels=False)

    # generation settings
    step_size = 10 # skip every n points
    plot_interval = 50 # n datapoints before current to show in graph

    # Generate and store plot images
    for i in tqdm(range(plot_interval, len(df), step_size)): 
        df = generate_imgs_for_training(df, i, plot_interval, pid=pid, dirname=dirname)

    # Add df to df_all dataframe
    if df_all.empty: df_all = df
    else: df_all = pd.concat([df_all, df], ignore_index=True)


# save the dataframe
df_all.to_csv("data.csv")

  0%|          | 0/980 [00:00<?, ?it/s]

100%|██████████| 980/980 [06:16<00:00,  2.60it/s]  


In [38]:
df_all.dropna(axis=0)

,time,glu_raw,glu,glu_d,glu_t,hr_raw,hr,hr_d,hr_t,iob_raw,...,bolus,done,basal_binned,bolus_binned,time_str,glu_url,hr_url,iob_url,basal_url,bolus_url
160,2021-12-07 13:20:00,206.0,0.557703,-0.581471,-1.257560,73.0,-0.038840,0.182794,-1.276998,8.311250,...,-0.099243,0.0,49.0,1.0,13:20:00,./img_data/559_160_glu.png,./img_data/559_160_hr.png,./img_data/559_160_iob.png,./img_data/559_160_basal.png,./img_data/559_160_bolus.png
170,2021-12-07 14:10:00,154.0,-0.179231,-1.172351,-2.023539,72.0,-0.102367,-1.186313,0.289424,5.886852,...,-0.099243,0.0,49.0,1.0,14:10:00,./img_data/559_170_glu.png,./img_data/559_170_hr.png,./img_data/559_170_iob.png,./img_data/559_170_basal.png,./img_data/559_170_bolus.png
180,2021-12-07 15:00:00,114.0,-0.746103,0.009410,-0.540128,70.0,-0.229421,0.274068,-0.060717,3.754006,...,-0.099243,0.0,49.0,1.0,15:00:00,./img_data/559_180_glu.png,./img_data/559_180_hr.png,./img_data/559_180_iob.png,./img_data/559_180_basal.png,./img_data/559_180_bolus.png
190,2021-12-07 15:50:00,98.0,-0.972852,0.157130,-0.464609,59.0,-0.928218,-0.182301,-0.742572,2.439053,...,-0.099243,0.0,49.0,1.0,15:50:00,./img_data/559_190_glu.png,./img_data/559_190_hr.png,./img_data/559_190_iob.png,./img_data/559_190_basal.png,./img_data/559_190_bolus.png
200,2021-12-07 16:40:00,100.0,-0.944508,-0.286031,-0.189503,65.0,-0.547056,0.182794,-0.558287,2.250000,...,-0.099243,0.0,49.0,1.0,16:40:00,./img_data/559_200_glu.png,./img_data/559_200_hr.png,./img_data/559_200_iob.png,./img_data/559_200_basal.png,./img_data/559_200_bolus.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9800,2022-01-10 00:40:00,89.0,-1.100398,-0.138310,-0.173320,63.0,-0.674110,0.000246,-0.014646,2.656301,...,-0.099243,0.0,36.0,1.0,00:40:00,./img_data/559_9800_glu.png,./img_data/559_9800_hr.png,./img_data/559_9800_iob.png,./img_data/559_9800_basal.png,./img_data/559_9800_bolus.png
9810,2022-01-10 01:30:00,82.0,-1.199600,-0.138310,-0.291993,64.0,-0.610583,0.000246,0.178853,2.250771,...,-0.099243,0.0,36.0,1.0,01:30:00,./img_data/559_9810_glu.png,./img_data/559_9810_hr.png,./img_data/559_9810_iob.png,./img_data/559_9810_basal.png,./img_data/559_9810_bolus.png
9820,2022-01-10 02:20:00,84.0,-1.171257,0.009410,0.123362,61.0,-0.801164,-0.091028,0.031425,1.927145,...,-0.099243,0.0,36.0,1.0,02:20:00,./img_data/559_9820_glu.png,./img_data/559_9820_hr.png,./img_data/559_9820_iob.png,./img_data/559_9820_basal.png,./img_data/559_9820_bolus.png
9830,2022-01-10 03:10:00,80.0,-1.227944,0.009410,-0.103195,66.0,-0.483529,0.182794,0.114354,1.727199,...,-0.099243,0.0,36.0,1.0,03:10:00,./img_data/559_9830_glu.png,./img_data/559_9830_hr.png,./img_data/559_9830_iob.png,./img_data/559_9830_basal.png,./img_data/559_9830_bolus.png


In [31]:
#df.at[9490, "hr"]
len(df[df.glu_raw.isna() == False])

728